In [3]:
import re
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction import text
from nltk.corpus import stopwords
#import english words
from nltk.corpus import words
import pandas as pd
import numpy as np

In [1]:

df = pd.read_csv('../data/processed/mimic_iii/diagnoses_noteevents.csv')

In [2]:
#show max length of text
max_len = df['TEXT'].str.len().max()
#print(max_len)
# max width of column setting
pd.set_option('display.max_colwidth', None)
#df.head()


In [ ]:

#headers
# find headers for all df['TEXT'] and count occurance
headers = {}
for i in range(len(df)):
    text = df['TEXT'][i]
    #clean text from \n
    text = text.replace('\n', ' ')
    #change tyoe to string
    text = str(text)
    header = re.findall(r'([A-Z][a-z]+ [A-Z][a-z]+:)', text)
    for h in header:
        if h in headers:
            headers[h] += 1
        else:
            headers[h] = 1
# sort headers by occurance
headers_sorted = {k: v for k, v in sorted(headers.items(), key=lambda item: item[1], reverse=True)}
# show headers in dataframe
headers_df = pd.DataFrame.from_dict(headers_sorted, orient='index', columns=['count'])
#list all unique headers
print(len(headers_df.index.tolist()))
headers_df.head()

#Headers to remove
#Headers to keep 
#History of Present Illness:
#Impression:
#ROS:
#Brief Hospital Course:
#HISTORY OF PRESENT ILLNESS:
#HISTORY:
#Respiratory:
#CHIEF COMPLAINT:



In [4]:
#THIS ONE

def keep_only_text_from_choosen_headers(text, choosen_headers):
    #print(text)
    new_text = ''
    #headers is a list of headers to keep
    
    text = str(text)
    #get all headers with regex (\n(.*?):\n)
    all_headers = re.findall(r'(\n(.*?):\n)', text)
    # get only first value in tuple
    all_headers = [x[1] for x in all_headers]

    #get the text after choosen headers until next header
    for c_h in choosen_headers:
        if c_h in text:
            if c_h in all_headers:
                #get index of choosen header
                index_c_h = all_headers.index(c_h)
                #get next header
                try:
                    next_header = all_headers[index_c_h+1]
                    #get index of next header
                    index_next_header = text.index(next_header)
                    text_after_c_h = text[text.index(c_h)+len(c_h):index_next_header]
                    #add text to new text
                    new_text += text_after_c_h
                except IndexError:
                    print('IndexError')
                    #raise NameError('IndexError')
                
    return new_text


 #apply function on complete dataframe
df['TEXT_final'] = df['TEXT'].apply(lambda x: keep_only_text_from_choosen_headers(x, ['CHIEF COMPLAINT', 'Chief Complaint', 'Present Illness','PRESENT ILLNESS',
                                                                                   'History of Present Illness',
                                                                                          'Impression', 
                                                                                          'ROS', 
                                                                                          'HISTORY OF PRESENT ILLNESS',
                                                                                          'Respiratory'
                                                                                        ]))



IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
# find most common words in text final which are not stop words

def most_common_words(series: pd.Series):
    #pd series of text to list of all words
    words = series.str.cat(sep=' ').split()
    stopwords_list = stopwords.words('english')
    # find most common words in text final which are not stop words
    #remove stop words
    words = [w for w in words if w not in stopwords_list]
    #create a dictionary with word and count
    words_dict = {}
    for w in words:
        if w in words_dict:
            words_dict[w] += 1
        else:
            words_dict[w] = 1
            
    #sort dictionary by count
    words_dict_sorted = {k: v for k, v in sorted(words_dict.items(), key=lambda item: item[1], reverse=True)}
    # get number of unique words
    unique_words =len(words_dict_sorted.keys())
    #create dataframe
    words_df = pd.DataFrame.from_dict(words_dict_sorted, orient='index', columns=['count'])
    return words_df 


df_most_common_words = most_common_words(df['TEXT_final'])       
#df_most_common_words.head(120)

        

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



,count
:,76578
He,70738
She,62111
patient,38309
.,35938
...,...
notable,5212
symptoms,5207
mild,5127
seen,5110


In [12]:
pd.set_option('display.max_rows', None)
df_most_common_words.head(200)
df_most_common_words.shape
# we have 214413 unique words so I will look at the top 0.1%
topwords = df_most_common_words.head(200)
personalised_stopwords_raw = topwords.index.tolist()
personalised_stopwords_raw

,count
:,76578
He,70738
She,62111
patient,38309
.,35938
pain,27440
**],25852
[**Hospital1,24858
The,23705
chest,22426


In [13]:
topwords = df_most_common_words.head(200)
personalised_stopwords_raw = topwords.index.tolist()
personalised_stopwords_raw

[':',
 'He',
 'She',
 'patient',
 '.',
 'pain',
 '**]',
 '[**Hospital1',
 'The',
 'chest',
 'given',
 'showed',
 'also',
 'In',
 'On',
 'denies',
 'history',
 'found',
 'transferred',
 'ED',
 'left',
 'Patient',
 'Name',
 'pain,',
 'noted',
 's/p',
 'started',
 'prior',
 '18**]',
 'right',
 'abdominal',
 'admitted',
 'CT',
 'Pt',
 '2',
 'presented',
 'IV',
 'reports',
 'pt',
 'recent',
 'last',
 'received',
 'No',
 'pain.',
 'BP',
 'ED,',
 'year',
 'old',
 '[**Known',
 'past',
 '1',
 'days',
 'lastname',
 'His',
 'blood',
 'OSH',
 'shortness',
 'arrival',
 'time',
 '[**Last',
 'yo',
 'This',
 'presents',
 'well',
 '[**Hospital',
 'HR',
 'male',
 'mg',
 'x',
 'day',
 'Her',
 'admission',
 'without',
 'At',
 'home',
 'felt',
 'initial',
 'developed',
 'revealed',
 '(un)',
 'acute',
 '3',
 'since',
 'back',
 'placed',
 'increased',
 'per',
 'A',
 'h/o',
 'recently',
 'CXR',
 'Per',
 'cardiac',
 'severe',
 'significant',
 'female',
 'head',
 'treated',
 'w/',
 'transfer',
 'L',
 'underwent

In [14]:
personalized_stopwords_filtered = [
 'He',
 'She',
 'patient',
 '**]',
 '[**Hospital1',
 'The',
 'given',
 'showed',
 'also',
 'In',
 'On',
 'denies',
 'history',
 'found',
 'transferred',
 'ED',
 'Patient',
 'Name',
 'noted',
 's/p',
 'started',
 'prior',
 '18**]',
 'admitted',
 'CT',
 'Pt',
 '2',
 'presented',
 'IV',
 'reports',
 'pt',
 'recent',
 'last',
 'received',
 'No',
 'BP',
 'ED,',
 'year',
 'old',
 '[**Known',
 'past',
 '1',
 'days',
 'lastname',
 'His',
 'OSH',
 'arrival',
 'time',
 '[**Last',
 'yo',
 'This',
 'presents',
 'well',
 '[**Hospital',
 'HR',
 'male',
 'mg',
 'x',
 'day',
 'Her',
 'admission',
 'without',
 'At',
 'home',
 'felt',
 'initial',
 'developed',
 'revealed',
 '(un)',
 '3',
 'since',
 'placed',
 'increased',
 'per',
 'A',
 'h/o',
 'recently',
 'CXR',
 'Per',
 'severe',
 'significant',
 'treated',
 'w/',
 'transfer',
 'L',
 'underwent',
 'initially',
 '[**Hospital3',
 'due',
 'states',
 'Denies',
 'one',
 'R',
 'notable',
 'symptoms',
 'seen',
 'ED.',
 'O2',
 'called',
 'RR',
 'status',
 'EKG',
 'several',
 'review',
 'Of',
 'feeling',
 'continued',
 'fevers,',
 'hospital',
 '[**Location',
 '(NI)',
 'Mr.',
 'went',
 'HTN,',
 'T',
 '(STitle)',
 'note,',
 'today',
 'VS',
 'became',
 'discharged',
 'MICU',
 'weeks',
 'ago',
 'episode',
 '4',
 'taken',
 'new',
 'sent',
 'normal',
 '[**Name',
 'medical',
 'episodes',
 'two',
 'chills,',
 'aortic',
 '100%',
 'denied',
 'improved',
 'possible',
 'unable',
 'SOB',
 'EMS',
 'morning',
 'associated',
 'elevated',
 'large',
 'reported',
 'brought',
 'week',
 '[**First',
 'RA.',
 'night',
 'course',
 'Dr.',
 'M',
 'GI',
 'decreased',
 'ICU',
 'WBC']

In [16]:
def clean_text(text):
    #replace \n with space
    text = text.replace('\n', ' ')
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(text))
    # Remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    #Remove digits
    document = re.sub(r'\d+', '', document)
    # Remove everything in brackets
    document = re.sub(r'\[[^]]*\]', '', document)
    # Filter out stop words
    document = ' '.join([word for word in document.split() if word not in stopwords.words('english')])
    # Filter out most common words
    document = ' '.join([word for word in document.split() if word not in personalized_stopwords_filtered])
    document = ' '.join([word for word in document.split() if len(word) > 1])
    return document   

#test function
df['TEXT_final_cleaned'] = df['TEXT_final'].apply(clean_text)
df.head()


TEXT  \
0  Admission Date:  [**2141-9-18**]              Discharge Date:   [**2141-9-24**]\n\nDate of Birth:  [**2117-8-7**]             Sex:   F\n\nService: MEDICINE\n\nAllergies:\nPenicillins / Percocet\n\nAttending:[**First Name3 (LF) 1936**]\nChief Complaint:\nhip pain\n\nMajor Surgical or Invasive Procedure:\nNone\n\n\nHistory of Present Illness:\n24yo woman with hx SLE, CKD(not currently on HD or PD), labile\nHTN here with right leg pain and HTN urgency. Patient was\nrecently d/ced on [**9-14**] following admission for the same\ncomplaints.\n.\nPatient took her hydralazine dose on am of admission. BP at\npresentation to the ER was 250/140 (The patient reportedly has\nbaseline SBPs in 130-170s) She was given 900 labetolol and 50\nhydralazine in the ED. BP following this was 175/124. Her EKG\nwas unchanged. K was 5.7.\n.\nPatient also complaining of [**10-20**] right hip pain. Patient was\nd/ced on [**9-14**] with oral dilaudid for hip/leg pain. She took this\nonly for one day due to severe itching. Denies any\nparasthesias/weakness. Her RLE/hip pain has been extensively\nworked up with negative LENIs, Lumbar spine MRI and hip plain\nfilms in the past. She was given 4mg IV morphine in the ER.\n.\nOn admission to the floor, leg/hip pain somwhat improved with\nmorphine.She denies any headache, vision changes, double vision,\nchest pain or SOB. Feels warm but no chills.\n\n\nPast Medical History:\n Systemic lupus erythematosus. Diagnosed [**2134**] (16 years old)\nwhen she had swollen fingers, arm rash and arthralgias. Previous\n\ntreatment with cytoxan, cellcept; currently on prednisone.\nComplicated by uveitis ([**2139**]) and ESRD ([**2135**]).\n- CKD/ESRD. Diagosed [**2135**]. Initiated dialysis [**2137**]. PD catheter\n\nplacement [**5-18**]. Pt reluctant to start PD.\n- Malignant hypertension. Baseline BPs 180's - 120's. History of\n\nhypertensive crisis with seizures. History of two\nintraparenchymal hemorrhages that were thought to be due to the\n\nposterior reversible leukoencephalopathy syndrome.\n- Thrombocytopenia. TTP (got plasmapheresisis) versus malignant\n\nHTN.\n- Thrombotic events. SVC thrombosis ([**2139**]); related to a\ncatheter. Negative lupus anticoagulant ([**4-/2138**], [**8-/2138**], 9/[**2140**]).\n\nNegative anticardiolipin antibodies IgG and IgM x4 ([**2137**]-[**2140**]).\n\nNegative Beta-2 glycoprotein antibody ([**4-/2138**], 8/[**2140**]).\n- HOCM: Last noted on echo [**8-17**].\n- Anemia.\n- History of left eye enucleation [**2139-4-20**] for fungal infection.\n\n- History of vaginal bleeding [**2139**] lasting 2 months s/p\nDepoProvera\ninjection requiring transfusion.\n- History of Coag negative Staph bacteremia and HD line\ninfection - [**6-16**] and [**5-17**]\n\n\nSocial History:\nSingle.  Recently moved into her own apartment. On disability.\nDenies EtOH, tobacco or recreational drug use.\n\n\nFamily History:\nNegative for autoimmune diseases, thrombophilic disorders.\nMaternal grandfather with HTN, MI, stroke in 70s.\n\n\nPhysical Exam:\nVS: T 97.3 HR 81 BP 135/91 RR 16 100%RA\nGen:NAD, happy, pleasant female\nHEENT:NC/AT, L eye prosthesis, R PERRL, EOMI, MMM, facial\nswelling, L side>R, scerla anicteric\nNeck: supple, no JVD, no LAD, scars from prior CVL and HD lines\nCV:S1S2+, 2/6 SEM LUSB, loud P2, RRR, no ectopy\nPulm: CTA B/L, good inspiratory effort\nAbd: +BS, soft, nontender, slightly distended and resonant to\npercussion, PD catheter in place in left abdomen\next: no c/c/e, 2+DP b/L, L foream swollen>R, R hip is mobile\nwithout pain elicited on passive or active movement\nneuro: AAOx3, nonfocal exam, CN 2-12 intact, moves all 4\nextremities\npsych: mood/affect appropriate\n\n\nPertinent Results:\nMRI Right Hip ([**9-18**]): There is no signal abnormality on the\nfluid-sensitive\nsequences in the proximal femur, acetabulum, or other osseous\nstructure. On T1- weighted images, there is a focal rounded\nregion within the right femoral head measuring approximately 9\nmm in a subch

In [17]:
# save Dataframe to csv file
df.to_csv("../data/processed/mimic_iii/diagnoses_noteevents_cleaned.csv", index=False)

In [19]:
#print len of text before and after cleaning
print('len of text before cleaning', len(df['TEXT'][0]))
print('len of text after cleaning', len(df['TEXT_final_cleaned'][0]))

len of text before cleaning 18402
len of text after cleaning 497


In [20]:
# print min , max, mean length of text before and after cleaning
print('min len of text before cleaning', df['TEXT'].str.len().min())
print('max len of text before cleaning', df['TEXT'].str.len().max())
print('mean len of text before cleaning', df['TEXT'].str.len().mean())
print('min len of text after cleaning', df['TEXT_final_cleaned'].str.len().min())
print('max len of text after cleaning', df['TEXT_final_cleaned'].str.len().max())
print('mean len of text after cleaning', df['TEXT_final_cleaned'].str.len().mean())
    

min len of text before cleaning 215.0
max len of text before cleaning 55728.0
mean len of text before cleaning 10391.011873601154
min len of text after cleaning 0
max len of text after cleaning 8719
mean len of text after cleaning 404.47100748358196
